### Different ways of reading and parsing data

#### Unzip the files

In [4]:
import os,zipfile
dir_name = "/data1/call_miner/Vinyas_Call_miner/call_miner"
os.chdir(dir_name)
zip_ref = zipfile.ZipFile('Vail_CDR_Transcripts_Jan18.zip') # create zipfile object
%time zip_ref.extractall('/data1/call_miner/Vinyas_Call_miner/call_miner/Jan_unzip') # extract file to dir
        #zip_ref.close() # close file

CPU times: user 6.1 s, sys: 4.89 s, total: 11 s
Wall time: 11 s


#### Extract nested zipped files 

In [6]:
import os,zipfile
dir_name = '/data1/call_miner/Vinyas_Call_miner/call_miner/Jan_unzip'
extension = ".zip"

os.chdir(dir_name) # change directory from working dir to dir with files

for item in os.listdir(dir_name): # loop through items in dir
    if item.endswith(extension): # check for ".zip" extension
        file_name = os.path.abspath(item) # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall('/data1/call_miner/Vinyas_Call_miner/call_miner/Jan_xml') #

#### Apply fix to xml - use this to fix the parsing error

In [7]:
#os.chdir("/data1/call_miner/Vinyas_Call_miner/call_miner/Feb_xml_unzipped")
import os, fnmatch
def findReplace(directory, find, replace, filePattern):
    for path, dirs, files in os.walk(os.path.abspath(directory)):
        for filename in fnmatch.filter(files, filePattern):
            filepath = os.path.join(path, filename)
            with open(filepath) as f:
                s = f.read()
            s = s.replace(find, replace)
            with open(filepath, "w") as f:
                f.write(s)

findReplace("/data1/call_miner/Vinyas_Call_miner/call_miner/Feb_xml", "</segment>\n<transcript>", "</segment>\n</transcript>", "*.xml")                
                

In [45]:
#Fix 01 : - Replace - "</segment>" --> "</segment>\n</transcript>"

#os.chdir("/data1/call_miner/Vinyas_Call_miner/call_miner/Feb_xml_unzipped")
import os, fnmatch
def findReplace(directory, find, replace, filePattern):
    for path, dirs, files in os.walk(os.path.abspath(directory)):
        for filename in fnmatch.filter(files, filePattern):
            filepath = os.path.join(path, filename)
            with open(filepath) as f:
                s = f.read()
            s = s.replace(find, replace)
            with open(filepath, "w") as f:
                f.write(s)

findReplace("/data1/call_miner/Vinyas_Call_miner/call_miner/May_xml_unzipped", "</segment>", "</segment>\n</transcript>", "*.xml")                
                

In [46]:
#Fix 02 : correct double replacemnts

#os.chdir("/data1/call_miner/Vinyas_Call_miner/call_miner/Feb_xml_unzipped")
import os, fnmatch
def findReplace(directory, find, replace, filePattern):
    for path, dirs, files in os.walk(os.path.abspath(directory)):
        for filename in fnmatch.filter(files, filePattern):
            filepath = os.path.join(path, filename)
            with open(filepath) as f:
                s = f.read()
            s = s.replace(find, replace)
            with open(filepath, "w") as f:
                f.write(s)

findReplace("/data1/call_miner/Vinyas_Call_miner/call_miner/May_xml_unzipped", "</segment>\n</transcript>\n</transcript>", "</segment>\n</transcript>", "*.xml")                
                

#### Extract Text from XML

In [ ]:
#Extract both Customer and Agent channel data

import unicodecsv as csv
import xml.etree.ElementTree as ET
import glob, os
import io
import sys

#from __future__ import print_function
#log = open("/data1/call_miner/Vinyas_Call_miner/call_miner/mar_output.txt", "w")
#sys.stdout = log  -- saves print output to a log file ,uncomment to track error file 

# Set the working directory
os.chdir("/data1/call_miner/Vinyas_Call_miner/call_miner/Sep_xml")
for filename in glob.glob('*.xml'):
                 #print ("parsing file",filename)  -- uncomment to track error file 
                 tree = ET.parse(filename,ET.XMLParser(encoding='Latin1')) 
                 root=tree.getroot()
                 eureka_id=root.find('eureka_id').text
                 with open("/data1/call_miner/Vinyas_Call_miner/call_miner/Sep_text_data/"+eureka_id+".txt","w") as f:
                        csvwriter = csv.writer(f)
                        for time in root.findall('segment'):
                            row=[]
                            task_name = time.find('words').text
                            row.append(task_name)
                            csvwriter.writerow(row)
                # print ('parsed...', filename) -- uncomment to track error file

                            
f.close()

#### Read Specific files - Exclude/Include files

In [51]:
import unicodecsv as csv
import glob, os
import io
from shutil import copyfile
import shutil

# Set the working directory
os.chdir("/data1/call_miner/Vinyas_Call_miner/call_miner/Sep_text_data")
filepath=''
# Open the csv file which has the list of call/eureka ids i.e. filenames that we wish to read and process
with open ("/data1/call_miner/Vinyas_Call_miner/call_miner/Lavanya/September/Sep_no_noise/sep_noise_id.csv") as f2:
    reader=csv.reader(f2)
    data_list=list(reader)
f2.close()    

data = [y for x in data_list for y in x]
[x.encode('UTF8') for x in data]

s = set(data)

for filename in glob.glob('*.txt'):
    if filename not in s:
        shutil.copy(filename,"/data1/call_miner/Vinyas_Call_miner/call_miner/Lavanya/September/Sep_denoise_text")

#### Get Call Duration and speaker volume - this derived dataset helps in identifying noise calls

In [1]:
import unicodecsv as csv
import xml.etree.ElementTree as ET
import glob, os
import io
import pandas as pd
from __future__ import division
import time

#from __future__ import print_function
#log = open("/data1/call_miner/Vinyas_Call_miner/call_miner/mar_output.txt", "w")

# Set the working directory
os.chdir("/data1/call_miner/Vinyas_Call_miner/call_miner/Sep_xml")
eureka_id=[]
duration=[]
silence=[]
wc=[]
cust=[]
agent=[]

for filename in glob.glob('*.xml'):
                 #print ("parsing file",filename)   
                 tree = ET.parse(filename,ET.XMLParser(encoding='Latin1')) 
                 root=tree.getroot()
                 eureka_id.append(root.find('eureka_id').text)
                 #print root.find('wav_length_sec').text   
                 duration.append (int(root.find('wav_length_sec').text))
                 silence.append (int(root.find('total_silence_sec').text))
                 #print root.find('total_silence_sec').text
                 wc.append(int(root.find('word_count').text))
                 c=0 # initialize the counter
                 a=0
                 for time in root.findall('segment'):
                            if time.find('speaker').text=='Customer':                        
                                c=c+1
                                #print(c)
                            else: a=a+1
                 #print(a,c)             
                 cust.append(c) 
                 agent.append(a)
                                            
                            
                                

data={'Eureka_id':eureka_id,'Call_duration':duration,'silence':silence,'word_count':wc,'cust':cust,'agent':agent}                 
                 
df_facts=pd.DataFrame(data)                   

df_facts.to_csv("/data1/call_miner/Vinyas_Call_miner/call_miner/Lavanya/September/Sep_no_noise/Sep_call_spr_cnt.csv")

In [2]:
import pandas as pd
df_facts=pd.read_csv("/data1/call_miner/Vinyas_Call_miner/call_miner/Lavanya/September/Sep_no_noise/Sep_call_spr_cnt.csv")

#df_facts=pd.read_csv("/data1/call_miner/Vinyas_Call_miner/call_miner/Lavanya/April/Noise_calls_Apr/Apr_call_spr_cnt.csv")

In [3]:
df_facts.dtypes

Unnamed: 0       int64
Call_duration    int64
Eureka_id        int64
agent            int64
cust             int64
silence          int64
word_count       int64
dtype: object

In [4]:
df_facts.shape

(871657, 7)

In [4]:
df_facts.head(5)

,Call_duration,Eureka_id,agent,cust,silence,word_count
0,115,16862366,8,8,41,238
1,105,16862374,16,17,25,286
2,274,16862376,26,26,109,572
3,155,16862372,10,9,78,256
4,277,16862382,32,31,101,615


#### Check various noise calls based on conditions 

In [ ]:
import pandasql as pdsql
pysql = lambda q: pdsql.sqldf(q, globals())

In [47]:
qry1 ="select * from df_facts where cust>3 and agent>3 and word_count<=150 and silence >=200;"

In [48]:
df3=pysql(qry1)

In [49]:
df3.shape # gets rows,columns

(44, 7)

In [50]:
df3.to_csv('/data1/call_miner/Vinyas_Call_miner/call_miner/Lavanya/September/Sep_no_noise/c_a_sil.csv')

In [14]:
df3.head(10)

,Unnamed: 0,Call_duration,Eureka_id,agent,cust,silence,word_count
0,352,109,15362340,1,0,79,100
1,471,126,15359057,1,0,95,91
2,567,33,15360494,1,0,13,74
3,649,80,15324031,1,0,44,119
4,704,83,15351018,1,0,37,169
5,714,74,15323320,1,0,34,124
6,1183,29,15358068,1,0,7,64
7,1381,68,15355402,1,0,29,142
8,1466,67,15349229,1,0,48,45
9,1516,54,15332066,1,0,49,16


#### Extract Customer or Agent channel text

In [3]:
#Extract only Customer channel data

import unicodecsv as csv
import xml.etree.ElementTree as ET
import glob, os
import io

# Set the working directory
os.chdir("/data1/call_miner/Vinyas_Call_miner/call_miner/Oct_xml")
for filename in glob.glob('*.xml'):
                 tree = ET.parse(filename,ET.XMLParser(encoding='Latin1'))
                 root=tree.getroot()
                 eureka_id=root.find('eureka_id').text
                 with open("/data1/call_miner/Vinyas_Call_miner/call_miner/Lavanya/Sentiment_Model/Oct_cust_text/"+eureka_id+".txt","w") as f:
                        csvwriter = csv.writer(f)
                        for time in root.findall('segment'):
                            row=[]
                            if time.find('speaker').text=='Customer':
                                task_name = time.find('words').text
                                row.append(task_name)
                                csvwriter.writerow(row)
                            
f.close()

In [5]:
#Extract only Agent channel data

import unicodecsv as csv
import xml.etree.ElementTree as ET
import glob, os
import io

# Set the working directory
os.chdir("/data1/call_miner/Vinyas_Call_miner/call_miner/Feb_xml_unzipped")
for filename in glob.glob('*.xml'):
                 tree = ET.parse(filename,ET.XMLParser(encoding='Latin1'))
                 root=tree.getroot()
                 eureka_id=root.find('eureka_id').text
                 with open("/data1/call_miner/Vinyas_Call_miner/call_miner/Feb_Cust_words/file_"+eureka_id+".txt","w") as f:
                        csvwriter = csv.writer(f)
                        for time in root.findall('segment'):
                            row=[]
                            if time.find('speaker').text=='Agent':
                                task_name = time.find('words').text
                                row.append(task_name)
                                csvwriter.writerow(row)
                            
f.close()

#### Read & Write data from HDFS

In [1]:
import subprocess
 
 
def run_cmd(args_list):
    print('Running system command: {0}'.format(' '.join(args_list)))
    proc = subprocess.Popen(args_list, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    s_output, s_err = proc.communicate()
    s_return = proc.returncode
    return s_return, s_output, s_err
(ret, out, err)= run_cmd(['hdfs', 'dfs', '-ls', 'hdfs_file_path'])
lines = out.split('\n')
 
 

Running system command: hdfs dfs -ls hdfs_file_path


In [16]:
#Run Hadoop put command in Python
(ret, out, err)= run_cmd(['hdfs', 'dfs', '-put', '/data1/call_miner/Vinyas_Call_miner/call_miner/syf_Vail_CDR_Transcripts_Sep18.zip', '/hpoc/edl/data/common/operations/callminer_data/call_miner'])
 

Running system command: hdfs dfs -put /data1/call_miner/Vinyas_Call_miner/call_miner/syf_Vail_CDR_Transcripts_Sep18.zip /hpoc/edl/data/common/operations/callminer_data/call_miner


In [10]:
#Run Hadoop get command in Python
(ret, out, err)= run_cmd(['hdfs', 'dfs', '-get', '/hpoc/edl/data/common/operations/callminer_data/call_miner/Vail_CDR_Transcripts_Jan18.zip', '/data1/call_miner/Vinyas_Call_miner/call_miner'])

Running system command: hdfs dfs -get /hpoc/edl/data/common/operations/callminer_data/call_miner/Vail_CDR_Transcripts_Jan18.zip /data1/call_miner/Vinyas_Call_miner/call_miner


#### Other data format - Merge multiple text files in an folder to single text file

In [3]:
import glob
import os
# Set the working directory
os.chdir("/data1/call_miner/Vinyas_Call_miner/call_miner/Feb_Walmart")
path='/data1/call_miner/Vinyas_Call_miner/call_miner/Feb_walmart_data.txt'
filenames = glob.glob('*.txt') 

with open(path, 'w') as f:
    for file in filenames:
        with open(file) as infile:
            f.write(infile.read()+'\n')           
   

In [3]:
#Use for this Data Format - Vail_CDR_Transcripts_Apr18_Corrected -> syf_2018_04_01.zip-> syf_2018_04_01->15759300.xml

import shutil
import os

RootDir1 = "/data1/call_miner/Vinyas_Call_miner/call_miner/Apr_xml"
TargetFolder = "/data1/call_miner/Vinyas_Call_miner/call_miner/Apr_xml_unzip"
for root, dirs, files in os.walk((os.path.normpath(RootDir1)), topdown=False):
        for name in files:
            if name.endswith('.xml'):
                #print "Found"
                SourceFolder = os.path.join(root,name)
                shutil.copy2(SourceFolder, TargetFolder)

#### Merge multiple files

In [ ]:
import glob
import os
# Set the working directory
os.chdir("/data1/call_miner/Vinyas_Call_miner/call_miner")
path='/data1/call_miner/Vinyas_Call_miner/call_miner/Feb_Dec_9_decile.txt'
filenames = ['Dec_9_decile_merged.txt','Feb_9_decile_merged.txt'] 

with open(path, 'w') as f:
    for file in filenames:
        with open(file) as infile:
            f.write(infile.read()+'\n')  